In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import lightgbm as lgb

In [2]:
base_dir = "data/"

In [3]:
# kurupical
df1 = pd.read_csv(f"{base_dir}/model275/epoch11.csv")
# df2 = pd.read_csv(f"{base_dir}/model276/epoch10.csv")
df3 = pd.read_csv(f"{base_dir}/model277/epoch11.csv")
df4 = pd.read_csv(f"{base_dir}/model277_2/epoch11.csv")
df5 = pd.read_csv(f"{base_dir}/model288_3/epoch19.csv")
df6 = pd.read_csv(f"{base_dir}/model294/epoch18.csv")
df7 = pd.read_csv(f"{base_dir}/model288_4/epoch24.csv")

In [ ]:
# takoi-san
df_val_row = pd.read_feather("../input/riiid-test-answer-prediction/train_transformer_last2500k_only_row_id.feather")

def extract_val_row(path):
    df = pd.read_csv(path)
    return pd.merge(df, df_val_row, how="inner")

df101 = extract_val_row(f"{base_dir}/ex67_68_70_74_75_predict.csv/ex67_68_70_74_75_predict.csv")
df102 = extract_val_row(f"{base_dir}/lgb014.csv/lgb014.csv") 
df103 = extract_val_row(f"{base_dir}/ex82_predict.csv/ex82_predict.csv")
df104 = extract_val_row(f"{base_dir}/ex85_predict.csv/ex85_predict.csv")

In [ ]:
df101.columns, df102.columns, df103.columns, df104.columns

In [ ]:
len(df101)

In [ ]:
# merge
df = pd.DataFrame()
df["target"] = df1["target"]
df["model275"] = df1["predict"]
# df["model276"] = df2["predict"]
df["model277"] = df3["predict"]
df["model277_2"] = df4["predict"]
df["model288_3"] = df5["predict"]
df["model294"] = df6["predict"]
df["model288_4"] = df7["predict"]
df["ex68"] = df101["ex68_transformer_75m"]
df["ex70"] = df101["ex70_transformer_75m"]
df["ex67"] = df101["ex67_lstm_75m"]
df["ex74"] = df101["ex74_transformer_conv1d_75m"]
df["ex75"] = df101["ex75_transformer_linear_75m"]
df["lgb014"] = df102["ex014_lgb"]
df["ex82"] = df103["ex82_transformer_75m"]
df["ex85"] = df104["ex85_transformer_75m"]

# df["ex55"] = df4["ex55"]
# df["ex56"] = df4["ex56"]
# df["ex57"] = df4["ex57"]

In [ ]:
df_train = df.iloc[:2_000_000]
df_val = df.iloc[2_000_000:]

In [ ]:
for col in df_val.columns:
    score = roc_auc_score(df_val["target"], df_val[col])
    print(f"model={col} auc={round(score, 4)}")

In [ ]:
params = {
    'objective': 'binary',
    'max_depth': -1,
    'num_leaves': 8,
    'learning_rate': 0.01,
    'bagging_fraction': 0.7,
    'bagging_seed': 0,
    'random_state': 0,
    'metric': 'auc',
    'verbosity': -1,
    "n_estimators": 10000,
    "early_stopping_rounds": 50
}

In [ ]:
# lgbm
train_data = lgb.Dataset(df_train.drop("target", axis=1), label=df_train["target"])
valid_data = lgb.Dataset(df_val.drop("target", axis=1), label=df_val["target"])

model = lgb.train(
    params,
    train_data,
    valid_sets=[train_data, valid_data],
    verbose_eval=100
)

In [ ]:
lgbm = model.predict(df_val.drop("target", axis=1))

In [ ]:
from datetime import datetime as dt
import os
import pickle
now = dt.now().strftime("%Y%m%d%H%M%S")
os.makedirs(f"model/{now}")
with open(f"model/{now}/lgbm_stacking014.pickle", "wb") as f:
    pickle.dump(model, f)

# 以下、没になったモデリング集

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import StackingClassifier

In [ ]:
clf = StackingClassifier(
    estimators= [
        ('lgb', lgb.LGBMClassifier()),
        ('et', ExtraTreesClassifier()),
        ('rf', RandomForestClassifier()),
        ('lr', LogisticRegression()),
    ],
    final_estimator=LogisticRegression()
)

In [ ]:
clf.fit(df_train.drop("target", axis=1).values, df_train["target"].values)

In [ ]:
pred = clf.predict_proba(df_val.drop("target", axis=1).values)[:, 1]

In [ ]:
# logistic reg
from sklearn.linear_model import LogisticRegression

clf_logistic = LogisticRegression()
clf_logistic.fit(df_train.drop("target", axis=1).values, df_train["target"].values)
logistic = clf_logistic.predict_proba(df_val.drop("target", axis=1).values)[:, 1]

In [ ]:
roc_auc_score(df_val["target"].values, logistic)

In [ ]:
# mlp
from sklearn.neural_network import MLPClassifier

clf_mlp = MLPClassifier()
clf_mlp.fit(df_train.drop("target", axis=1).values, df_train["target"].values)
mlp = clf_mlp.predict(df_val.drop("target", axis=1).values)

In [ ]:
mlp = clf_mlp.predict_proba(df_val.drop("target", axis=1).values)[:, 1]
roc_auc_score(df_val["target"].values, mlp)

In [ ]:
roc_auc_score(df_val["target"].values, lgbm*10 + logistic + mlp)

In [ ]:
import torch
import torch.nn.functional
import torch.utils.data


class Net(torch.nn.Module):
    def __init__(self, input_dim):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, 128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.fc3 = torch.nn.Linear(64, 1)
    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
num_epochs = 100
X_train = torch.from_numpy(df_train.drop("target", axis=1).values).float().to("cuda")
y_train = torch.from_numpy(df_train["target"].values.reshape(-1, 1)).float().to("cuda")

X_val = torch.from_numpy(df_val.drop("target", axis=1).values).float().to("cuda")
y_val = df_val["target"].values.reshape(-1, 1)

clf_nn = Net(input_dim=6).to("cuda")
clf_nn.train()

# set training parameters
optimizer = torch.optim.Adam(clf_nn.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()

# start to train
epoch_loss = []
for epoch in range(num_epochs):
    # forward
    outputs = clf_nn(X_train)
    
    # calculate loss
    loss = criterion(outputs, y_train)
    
    # update weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(roc_auc_score(y_val, clf_nn(X_val).data.cpu().numpy().tolist()))
    # save loss of this epoch